In [1]:
'''
noise complaint data taken from openNYC data portal
https://data.cityofnewyork.us/Social-Services/NYC-311-NYPD-Noise-2015-01-14/a75n-km7r

can only get 2015-2017 data for some reason
'''

'\nnoise complaint data taken from openNYC data portal\nhttps://data.cityofnewyork.us/Social-Services/NYC-311-NYPD-Noise-2015-01-14/a75n-km7r\n\ncan only get 2015-2017 data for some reason\n'

In [2]:
import numpy as np
import pandas as pd
import geopandas as gp
import os
import subprocess
import math
import shapely as shp
from geopandas import GeoDataFrame
from shapely.geometry import Point
from datetime import datetime
from fiona.crs import from_epsg
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import re
from pandas.tseries.holiday import get_calendar, HolidayCalendarFactory, GoodFriday
from datetime import datetime
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score as rs
from sklearn import svm, datasets
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

from itertools import compress

np.random.seed(222)

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [3]:
data_folder = '/Users/geoffperrin/Desktop/CUSP/GRA/NYPD/data/'

In [21]:
noise_df_raw = pd.read_csv(data_folder + '311_noise_2013_2015.csv')

/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (7,17,18,20,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
noise_df = noise_df_raw[['Created Date', 'Descriptor', 'Latitude', 'Longitude']]

In [24]:
noise_df = noise_df.dropna()

In [25]:
noise_df['lonlat'] = zip(noise_df.Longitude, noise_df.Latitude)
noise_df['geometry'] = noise_df[['lonlat']].applymap(lambda x:shp.geometry.Point(x))
noise_df = noise_df.drop(['Longitude', 'Latitude'], axis=1)
crs = {'init': 'epsg:4326'}
noise_df_geo = GeoDataFrame(noise_df, crs=crs, geometry = 'geometry')
noise_df_geo = noise_df_geo.to_crs(epsg=4326)

In [26]:
noise_df_geo = noise_df_geo[['Created Date', 'Descriptor', 'geometry']]
noise_df_geo.head()

,Created Date,Descriptor,geometry
0,05/01/2013 06:17:00 PM,"Noise, Barking Dog (NR5)",POINT (-73.8589886717692 40.89389930459917)
1,07/04/2015 01:02:00 PM,Noise: air condition/ventilation equipment (NV1),POINT (-73.95763793947506 40.71241137254676)
2,07/29/2015 02:00:00 AM,Noise: air condition/ventilation equipment (NV1),POINT (-73.90252924410468 40.7438245925335)
3,11/12/2015 08:59:00 AM,Noise: air condition/ventilation equipment (NV1),POINT (-73.90993751295274 40.81905560719026)
4,10/12/2015 01:07:00 AM,Noise: air condition/ventilation equipment (NV1),POINT (-73.94300888631271 40.61791057506885)


In [9]:
### read in census block GROUP shapefile
ny_cb_shp_raw = gp.read_file(data_folder + 'cb_2016_36_bg_500k/cb_2016_36_bg_500k.shp')
ny_cb_shp_raw = ny_cb_shp_raw.to_crs(epsg=4326)
ny_cb_shp_raw.head()

,AFFGEOID,ALAND,AWATER,BLKGRPCE,COUNTYFP,GEOID,LSAD,NAME,STATEFP,TRACTCE,geometry
0,1500000US360850156031,332908,0,1,085,360850156031,BG,1,36,015603,"POLYGON ((-74.16134900000002 40.529374, -74.15..."
1,1500000US361039901000,0,2712461516,0,103,361039901000,BG,0,36,990100,"POLYGON ((-72.18759507814521 41.0338154646346,..."
2,1500000US360050004001,62874,0,1,005,360050004001,BG,1,36,000400,"POLYGON ((-73.85870299999999 40.815405, -73.85..."
3,1500000US360594097001,374608,0,1,059,360594097001,BG,1,36,409700,"POLYGON ((-73.52014 40.698804, -73.51471599999..."
4,1500000US360593041002,875235,0,2,059,360593041002,BG,2,36,304100,"POLYGON ((-73.587903 40.760198, -73.5872079999..."


In [10]:
ny_cb_shp = ny_cb_shp_raw[(ny_cb_shp_raw.COUNTYFP == '081') | (ny_cb_shp_raw.COUNTYFP == '005') |\
                         (ny_cb_shp_raw.COUNTYFP == '047') | (ny_cb_shp_raw.COUNTYFP == '061') |\
                         (ny_cb_shp_raw.COUNTYFP == '085')]

ny_cb_shp = ny_cb_shp_raw[['AFFGEOID', 'GEOID', 'geometry']]

In [11]:
#ny_cb_shp.plot()

In [27]:
noise_cb_df = gp.sjoin(noise_df_geo, ny_cb_shp, how='left')

In [28]:
noise_cb_df.head()

,Created Date,Descriptor,geometry,index_right,AFFGEOID,GEOID
0,05/01/2013 06:17:00 PM,"Noise, Barking Dog (NR5)",POINT (-73.8589886717692 40.89389930459917),1762.0,1500000US360050420002,360050420002
1,07/04/2015 01:02:00 PM,Noise: air condition/ventilation equipment (NV1),POINT (-73.95763793947506 40.71241137254676),2341.0,1500000US360470523005,360470523005
2,07/29/2015 02:00:00 AM,Noise: air condition/ventilation equipment (NV1),POINT (-73.90252924410468 40.7438245925335),1930.0,1500000US360810249001,360810249001
3,11/12/2015 08:59:00 AM,Noise: air condition/ventilation equipment (NV1),POINT (-73.90993751295274 40.81905560719026),1862.0,1500000US360050075002,360050075002
4,10/12/2015 01:07:00 AM,Noise: air condition/ventilation equipment (NV1),POINT (-73.94300888631271 40.61791057506885),4832.0,1500000US360470746002,360470746002


In [29]:
noise_cb_df_groupby = noise_cb_df.groupby(['GEOID'], as_index=False)['Descriptor'].count()
noise_cb_df_groupby.head()

,GEOID,Descriptor
0,360050002001,150
1,360050002002,226
2,360050002003,80
3,360050004001,38
4,360050004002,124


In [30]:
ny_cb_shp_w_noise_counts = ny_cb_shp.merge(noise_cb_df_groupby, on='GEOID')

In [31]:
ny_cb_shp_w_noise_counts.head()

,AFFGEOID,GEOID,geometry,Descriptor
0,1500000US360850156031,360850156031,"POLYGON ((-74.16134900000002 40.529374, -74.15...",30
1,1500000US360050004001,360050004001,"POLYGON ((-73.85870299999999 40.815405, -73.85...",38
2,1500000US360811447001,360811447001,"POLYGON ((-73.77740299999999 40.758214, -73.77...",36
3,1500000US360810269013,360810269013,"POLYGON ((-73.88256200000001 40.747663, -73.87...",76
4,1500000US360470444001,360470444001,"POLYGON ((-73.969112 40.61701, -73.96774000000...",9


In [32]:
ny_cb_shp_w_noise_counts.to_csv(data_folder + 'ny_cb_shp_w_noise_counts')